### Checklist for submission

It is extremely important to make sure that:

1. Everything runs as expected (no bugs when running cells);
2. The output from each cell corresponds to its code (don't change any cell's contents without rerunning it afterwards);
3. All outputs are present (don't delete any of the outputs);
4. Fill in all the places that say `# YOUR CODE HERE`, or "**Your answer:** (fill in here)".
5. Never copy/paste any notebook cells. Inserting new cells is allowed, but it should not be necessary.
6. The notebook contains some hidden metadata which is important during our grading process. **Make sure not to corrupt any of this metadata!** The metadata may for example be corrupted if you copy/paste any notebook cells, or if you perform an unsuccessful git merge / git pull. It may also be pruned completely if using Google Colab, so watch out for this. Searching for "nbgrader" when opening the notebook in a text editor should take you to the important metadata entries.
7. Although we will try our very best to avoid this, it may happen that bugs are found after an assignment is released, and that we will push an updated version of the assignment to GitHub. If this happens, it is important that you update to the new version, while making sure the notebook metadata is properly updated as well. The safest way to make sure nothing gets messed up is to start from scratch on a clean updated version of the notebook, copy/pasting your code from the cells of the previous version into the cells of the new version.
8. If you need to have multiple parallel versions of this notebook, make sure not to move them to another directory.
9. Although not forced to work exclusively in the course `conda` environment, you need to make sure that the notebook will run in that environment, i.e. that you have not added any additional dependencies.

**FOR HA1, HA2 ONLY:** Failing to meet any of these requirements might lead to either a subtraction of points (at best) or a request for resubmission (at worst).

We advise you to perform the following steps before submission to ensure that requirements 1, 2, and 3 are always met: **Restart the kernel** (in the menubar, select Kernel$\rightarrow$Restart) and then **run all cells** (in the menubar, select Cell$\rightarrow$Run All). This might require a bit of time, so plan ahead for this (and possibly use a cloud GPU in HA1 and HA2 for this step). Finally press the "Save and Checkout" button before handing in, to make sure that all your changes are saved to this .ipynb file.

### Fill in name of notebook file
This might seem silly, but the version check below needs to know the filename of the current notebook, which is not trivial to find out programmatically.

You might want to have several parallel versions of the notebook, and it is fine to rename the notebook as long as it stays in the same directory. **However**, if you do rename it, you also need to update its own filename below:

In [208]:
nb_fname = "IHA2.ipynb"

### Fill in group number and member names (use NAME2 and GROUP only for HA1 and HA2):

In [209]:
NAME1 = "Mattia Carlino"
NAME2 = ""
GROUP = ""

### Check Python version

In [210]:
from platform import python_version_tuple

assert (
    python_version_tuple()[:2] == ("3", "11")
), "You are not running Python 3.11. Make sure to run Python through the course Conda environment."

### Check that notebook server has access to all required resources, and that notebook has not moved

In [211]:
import os

nb_dirname = os.path.abspath("")
assignment_name = os.path.basename(nb_dirname)
assert assignment_name in [
    "IHA1",
    "IHA2",
    "HA1",
    "HA2",
], "[ERROR] The notebook appears to have been moved from its original directory"

### Verify correct nb_fname

In [ ]:
from IPython.display import HTML, display

try:
    display(
        HTML(
            r'<script>if("{nb_fname}" != IPython.notebook.notebook_name) {{ alert("You have filled in nb_fname = \"{nb_fname}\", but this does not seem to match the notebook filename \"" + IPython.notebook.notebook_name + "\"."); }}</script>'.format(
                nb_fname=nb_fname
            )
        )
    )
except NameError:
    assert False, "Make sure to fill in the nb_fname variable above!"

### Verify that your notebook is up-to-date and not corrupted in any way

In [ ]:
import sys

sys.path.append("..")
from ha_utils import check_notebook_uptodate_and_not_corrupted

check_notebook_uptodate_and_not_corrupted(nb_dirname, nb_fname)

# IHA2 - Catching Pokemon

![](https://upload.wikimedia.org/wikipedia/en/4/46/Pokemon_Go.png)

In this home assignment, you'll apply roughly the same principles we used when doing logistic regression on the Iris dataset in Computer Lab 1, but on a different and very interesting dataset. We'll use the [Predict'em All dataset from Kaggle](https://www.kaggle.com/semioniy/predictemall). To download the dataset you will need a Kaggle account. This dataset consists of roughly 293,000 [pokemon](http://www.pokemongo.com/) sightings (historical appearances of Pokemon in the Pokemon Go game), with geographical coordinates, time, weather, population density, distance to pokestops/gyms etc. as features. A comprehensive list of all the features is available at [the dataset's homepage](https://www.kaggle.com/semioniy/predictemall).

The context is simple: you are a Pokemon hunter, and there are only three Pokemon left for you to complete your collection. You'll do anything to capture them, including changing where you'll spend your next holidays! You know that some Pokemon only spawn in certain places of the world. Since you like machine learning so much, you figure it would be a great idea to train a classifier that, based on a location's longitude and latitude, can tell us which Pokemon is more likely to appear there.

The assignment is broken down into six steps.

1. Loading the data and extracting the desired subset of it
2. Visualization of the dataset
3. Preprocessing
4. Training
5. Evaluation
6. Exploration

Feel free to temporarily add cells wherever you see fit, and play around with this notebook as much as you want when developing the solutions. However, the solution you upload to Canvas must have the exact format shown here, with only the cells present here.

Don't restrict yourself only to what was taught so far. Some of the tasks might require you to search for new information. However, **be sure that you do the assignment using PyTorch** since we will be using it through the following assignments as well. [The Python docs](https://docs.python.org/3/), [PyTorch docs](https://pytorch.org/docs/stable/index.html), [stackoverflow](https://stackoverflow.com/), and Google are your friends!

**Hint:** Solving Computer Lab 1 (CL1) is a good way to get prepared for this assignment.

To pass this assignment, your solutions should pass all tests (`assert`-statements). Note that the tests shown to you are not exhaustive, and additional hidden tests exist for some of the tasks. Further, similar to IHA1, this notebook contains some questions where we ask you to reflect upon some results. These questions will not be graded in detail, but we still expect you to answer them.

## 0. Imports

Import any necessary modules here.

In [214]:
# YOUR CODE HERE
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import torch 
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import matplotlib.patches as mpatches
from matplotlib.colors import ListedColormap


## 1. Loading and extracting subset

The first step consists of filtering the dataset by the three Pokémon you are interested at. 

Start by loading the `'300k.csv'` file using pandas. If you haven't downloaded it yet, either use [this link](https://www.kaggle.com/semioniy/predictemall) to do so (and place the file in the same folder as this notebook), or simply run the cell below. You might have to [create a new API token](https://www.kaggle.com/settings/account) before the commands work.

In [ ]:
!kaggle datasets download -d semioniy/predictemall
!unzip -u predictemall.zip
!rm -rf predictemall.zip 300k_arff 300k.arff 300k_csv

In [ ]:
# TODO: load the dataset using pandas to a dataframe called df
# YOUR CODE HERE
df = pd.read_csv('300k.csv')

In [217]:
assert df.shape == (
    296021,
    208,
), f"Dataframe has not the right shape. {df.shape} != (296021, 208)"
assert isinstance(df, pd.DataFrame), f"df is not a dataframe. Was {type(df)}"

Modify `df` to only have the columns `latitude`, `longitude`, and `class`.

In [218]:
# YOUR CODE HERE
df = df[['latitude', 'longitude', 'class']]

In [219]:
assert len(df.columns) == 3, "There should be 3 columns"
assert len(df.shape) == 2, "The dataframe should be 2 dimensional"
assert df.shape == (296021, 3), "Wrong shape of the dataframe"
assert "latitude" in df.columns, "latitude column is missing"
assert "longitude" in df.columns, "longitude column is missing"
assert "class" in df.columns, "class column is missing"

Note that the `class` column specifies which pokemon it is. However, it only has the numerical id of the pokemon. For your convenience, we provide the dictionary `name_dict` to convert between ids and names (we'll do this soon).

In [220]:
from utils import name_dict

In [ ]:
# example usage (you can index either by name or id)
print(name_dict["Gengar"])
print(name_dict[94])

In [ ]:
# for convenience, let's add a new column to the dataframe with the name of the pokemon
df["name"] = df["class"].apply(lambda x: name_dict[x])
print(df.head())

We are only interested in three specific pokemon: Diglett, Seel, and Tauros.

<table style="width:100%">
  <tr>
    <th> <center>Diglett</center> </th>
    <th> <center>Seel</center> </th> 
    <th> <center>Tauros</center> </th>
  </tr>
  <tr>
    <td><img src=https://assets.pokemon.com/assets/cms2/img/pokedex/full/050_f2.png alt=Digglet></td>
    <td><img src=https://pokemon.gamepedia.com/media/pokemon.gamepedia.com/thumb/f/f1/Seel.png/200px-Seel.png?version=2c32fbe0af2d0da707e5dbcb40472fbf></td>
    <td><img src=https://www.pokemon.com/static-assets/content-assets/cms2/img/pokedex/full/128.png></td>
  </tr>
</table>



Filter the dataset to contain only these three types of pokemon and save it in the DataFrame `filtered_df`.

In [ ]:
# YOUR CODE HERE
filtered_df = df.query('name == "Diglett" or name == "Seel" or name == "Tauros"')
print(
    f"We have {len(filtered_df)} instances of Diglett, Seel, and Tauros in the dataset."
)

In [224]:
assert len(np.unique(filtered_df["class"])) == 3, "There should be 3 unique classes."
assert filtered_df.shape == (
    2083,
    4,
), "The shape of the filtered dataframe is incorrect."

In an earlier cell, you could see that the dataset has 208 features per pokemon sighting (`df.shape == (296021, 208)`). Why do we only use the `longitude` and `latitude` features and not all the features? 

**Your answer:** Because we are interested where pokemons can appear

## 2. Visualization of the dataset

The second step consists of visualizing the dataset. This will help you understand the distribution of the features and get an idea of how hard the task will be.

Plot a bar chart of the number of occurrences of each class.

In [ ]:
# YOUR CODE HERE
fig, ax = plt.subplots()
pokemons = ['Diglett', 'Seel', 'Tauros']
occurences = filtered_df['class'].value_counts()

bar_labels = ['Diglett', 'Seel', 'Tauros']
bar_colors = ['tab:red', 'tab:blue', 'tab:orange']

ax.bar(pokemons, occurences, label=bar_labels, color=bar_colors)

ax.set_ylabel('pokemon occurences')
ax.set_title('Number of occurences of each class')
ax.legend(title='Pokemon class')

print('occ:', occurences)
plt.show()

Is the dataset balanced? Why/why not? Why is this important?


**Your answer:** The dataset is not exactly balanced, since each class has different occurences. Balanced Datasets ensure that the model treats all classes equally, allowing it to learn features from all classes more effectively

Plot a scatter plot where the first dimension is longitude, the second is latitude, and each point is a Pokemon. Further, the color of each point should represent which Pokemon it is. Lastly, the marker at each point should be an `'x'`. Make sure to label each axis.

Hints:

- The `scatter` method from `matplotlib` accepts an argument called `c`.
- The `scatter` method also accepts an argument called `marker`.

In [ ]:
# YOUR CODE HERE
colors = {'Diglett': '#ff9999',
          'Seel': '#99ccff',   
          'Tauros': '#ffcc99'}
for class_name, class_df_filtered in filtered_df.groupby('name'):
    plt.scatter(class_df_filtered['longitude'], class_df_filtered['latitude'], c=colors[class_name], label=class_name, marker='x')
plt.xlabel('Longitude')
plt.ylabel('Latitude')
plt.title('Scatter plot of Pokémon by Location')
plt.legend(title='Pokémon Class')
plt.show()

Is there any other visualization you think would be useful? If so, insert them here.

In [227]:
# YOUR CODE HERE

How hard do you think the problem is? Which classes can/cannot be easily separated?


**Your answer:** From the scatter plot, it seems like there are clusters of Pokemon classes in different regions. Seel and Diglett appear to be the easiest to separate as them points cluster distinctly from the others. However, Diglett and Tauros have overlapping regions, particularly in the top-left area of the plot, making them harder to distinguish.

Which accuracy do you expect to achieve?

**Your answer:** Due to the clear separation between Diglett and Seel clusters, we can expect to achieve high general accuracy. However, the overlapping area with Tauros may slightly can reduce the specific class accuracy. A reasonable estimate would be around 70-80% accuracy

## 3. Preprocessing

The third step consists of processing the data before training, such as dividing the dataset into training, validation, and test sets. Some tranformations can also be applied to the dataset in order to improve the performance of the network. We will use some PyTorch utilities to help us with this task.

Start by creating the input and output vectors, `x` and `y`. The input should be latitude and longitude, and the output should be the class of the pokemon. Note that you cannot use the class names directly, as they are strings. You need to introduce some mapping to convert them to integers (0, 1, and 2) or one-hot vectors.

In [ ]:
# YOUR CODE HERE
# Input vector
# Use the latitude and longitude as input features
x = filtered_df[['latitude', 'longitude']].to_numpy(dtype="float32")

#Output vector
pokemon_mapping = {'Diglett': 0, 'Seel': 1, 'Tauros': 2}
# Label encoding
y = filtered_df['name'].apply(lambda name: pokemon_mapping[name]).to_numpy()

print(f"Shape of input data: {x.shape}")
print(f"Shape of labels: {y.shape}")

In [229]:
assert isinstance(x, np.ndarray), "x should be a numpy array"
assert isinstance(y, np.ndarray), "y should be a numpy array"

assert x.shape[0] == y.shape[0], "x and y should have the same number of samples"
assert x.shape[-1] == 2, "x should have 2 features"
assert x.dtype == np.float32, "x should be of type float32"
if y.shape[-1] == 3:  # one-hot encoded
    assert y.max() == 1, "one-hot encoded y, at least one entry should be 1"
    assert y.min() == 0, "one-hot encoded y, at least one entry should be 0"
    assert y.sum(axis=1).all() == 1
else:  # label encoded
    assert y.max() == 2, "label encoded y, should have a max value of 2"
    assert y.min() == 0, "label encoded y, should have a min value of 0"

Separate your data into training (55%), validation (25%) and test sets (20%) and save them as `dataset_train`, `dataset_val`, `dataset_test`. If you wish to apply any transformation to the dataset, do it here as well. 

Further, create a class, PokemonDataset, inheriting from PyTorch [`Dataset`](https://pytorch.org/docs/stable/data.html#torch.utils.data.Dataset) and use this for storing the data. In other words, `dataset_train`, `dataset_val`, `dataset_test` should have type PokemonDataset. You will need to implement a `__getitem__`, `__len__` and `__init__` method. Although perhaps a bit overkill for this assignment, it is a good practice for handling datasets in PyTorch.

Last, instantiate a [`DataLoader`](https://pytorch.org/docs/stable/data.html#torch.utils.data.DataLoader) for each dataset, i.e., `loader_train`, `loader_val`, `loader_test`. This will fetch samples from the datasets and combine them into batches. Remember to select a suitable batch size.

In [ ]:
# YOUR CODE HERE
# separate dataset into train-val and test set
x_train_val, x_test, y_train_val, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

# separate into training and validation sets
x_train, x_val, y_train, y_val = train_test_split (x_train_val, y_train_val, test_size=0.3125, random_state=42)

print(f"Training set size: {len(x_train)}")
print(f"Validation set size: {len(x_val)}")
print(f"Test set size: {len(x_test)}")

# Standardize the data
scaler = StandardScaler()
x_train_scaled = scaler.fit_transform(x_train)
x_val_scaled = scaler.transform(x_val)
x_test_scaled = scaler.transform(x_test)

# Create our own Pokemon dataset
class PokemonDataset(Dataset):
    def __init__(self, x_data, y_data):
        self.x_data = x_data
        self.y_data = y_data
        
    def __len__(self):
        return len(self.x_data)
    
    def __getitem__(self, idx):
        sample = {'features': torch.tensor(self.x_data[idx], dtype=torch.float32),
              'label': torch.tensor(self.y_data[idx], dtype=torch.long)}
        return sample['features'], sample['label']

# Create the datasets    
dataset_train = PokemonDataset(x_train_scaled, y_train)
dataset_val = PokemonDataset(x_val_scaled, y_val)
dataset_test = PokemonDataset(x_test_scaled, y_test)

# Batch size
batch_size = 512

# Dataloader creation (datasets loaded into DataLoader)
loader_train = DataLoader(dataset_train, batch_size=batch_size, shuffle=True)
loader_val = DataLoader(dataset_val, batch_size=batch_size, shuffle=False)
loader_test = DataLoader(dataset_test, batch_size=batch_size, shuffle=False)


In [231]:
assert isinstance(
    dataset_train, PokemonDataset
), "dataset_train should be an instance of PokemonDataset"
assert isinstance(
    dataset_val, PokemonDataset
), "dataset_val should be an instance of PokemonDataset"
assert isinstance(
    dataset_test, PokemonDataset
), "dataset_test should be an instance of PokemonDataset"

assert (
    abs(len(dataset_train) / len(x) - 0.55) < 0.01
), "dataset_train has the wrong length, should be 55% of the data"
assert (
    abs(len(dataset_val) / len(x) - 0.25) < 0.01
), "dataset_val has the wrong length, should be 25% of the data"
assert (
    abs(len(dataset_test) / len(x) - 0.20) < 0.01
), "dataset_test has the wrong length, should be 20% of the data"

assert isinstance(
    loader_train, DataLoader
), "loader_train should be an instance of DataLoader"
assert isinstance(
    loader_val, DataLoader
), "loader_val should be an instance of DataLoader"
assert isinstance(
    loader_test, DataLoader
), "loader_test should be an instance of DataLoader"

assert len(loader_train), "loader_train should have a length"
assert len(loader_val), "loader_val should have a length"
assert len(loader_test), "loader_test should have a length"


## 4. Training

The fourth step is where you will choose the architecture of your network (number of hidden layers, activation functions, etc.), optimizer, loss function and then train the network. 

Start by implementing a training loop, and a helper function to calculate the accuracy. The training loop should calculate the loss and accuracy for both the training and validation set and print it with some regular interval (each epoch, or every few epochs, for instance). It can also be helpful to plot the loss and accuracy for both the training and validation set, either in the training loop, or after it has finished (i.e., you have to store the values during training and return them).

We have prepared the `train_model` function with the arguments you need, but you have access to *args and **kwargs if you want to pass additional arguments to the function.

In [251]:
from typing import Callable, Union

import torch
import torch.nn as nn
import torch.utils


def accuracy(y_hat: torch.Tensor, y: torch.Tensor) -> torch.Tensor:
    """
    Compute the accuracy of the model.

    Args:
    y_hat: torch.Tensor: The model predictions (probability per class), shape: [batch_size, num_classes]
    y: torch.Tensor: The true labels, shape: [batch_size]

    Returns:
    torch.Tensor: The accuracy of the model
    """
    # YOUR CODE HERE
    # Get the predicted class by taking the argmax over the class dimension
    prediction = torch.argmax(y_hat, dim=1)
    # Compute accuracy by comparing predictions with true labels
    acc = torch.tensor(prediction == y).float().mean()
    return acc


def train_model(
    model: nn.Module,
    optimizer: torch.optim,
    loss_fn: Union[Callable, nn.Module],
    num_epochs: int,
    train_dataloader: torch.utils.data.DataLoader,
    val_dataloader: torch.utils.data.DataLoader,
    device: Union[str, torch.device],
    *args,
    **kwargs,
):
    """
    Train the model.

    Args:
    model: nn.Module: The neural network model
    optimizer: torch.optim: The optimizer used to update the model parameters
    loss_fn: Union[Callable, nn.Module]: The loss function used to compute the loss
    num_epochs: int: The number of epochs to train the model
    train_dataloader: torch.utils.data.DataLoader: The training dataloader
    val_dataloader: torch.utils.data.DataLoader: The validation dataloader
    device: Union[str, torch.device]: The device to run the training on
    *args: Additional arguments to pass to the train function
    **kwargs: Additional keyword arguments to pass to the train function

    """

    # YOUR CODE HERE

    # Set the model to the device
    model.to(device)

    train_losses = []
    val_losses = []
    train_accuracies = []
    val_accuracies = []

    # Iterate over the epochs
    for epoch in range(num_epochs):
        # Set the model to train mode
        model.train()

        # Initialize running loss and accuracy for the current epoch
        current_train_loss = 0
        current_train_acc = 0
        
        for input_batch, labels_batch in train_dataloader:
            # Move the batch to the device
            input_batch, labels_batch = input_batch.to(device), labels_batch.to(device)

            # Zero the parameter gradients, to avoid accumulation of gradients in the optimizer 
            optimizer.zero_grad()

            # Compute forward pass (compute predictions)
            outputs = model(input_batch)
            # Compute loss
            loss = loss_fn(outputs, labels_batch)

            # Backward pass and optimization
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            # Update training loss and accuracy
            current_train_loss += loss.item()
            current_train_acc += accuracy(outputs, labels_batch).item()

        # Calculate average training loss and accuracy for the epoch
        current_train_loss /= len(train_dataloader)
        current_train_acc /= len(train_dataloader)
        train_losses.append(current_train_loss)
        train_accuracies.append(current_train_acc)

        # Set the model to evaluation mode
        model.eval()
        current_val_loss = 0
        current_val_acc = 0

        with torch.no_grad():
            for inputs_batch, labels_batch in val_dataloader:
                # Move data to device
                inputs_batch, labels_batch = inputs_batch.to(device), labels_batch.to(device)

                # Forward pass
                outputs = model(inputs_batch)
                loss = loss_fn(outputs, labels_batch)
                
                # Update validation loss and accuracy
                current_val_loss += loss.item()
                current_val_acc += accuracy(outputs, labels_batch).item()

        # Average validation loss and accuracy for the epoch
        current_val_loss /= len(val_dataloader)
        current_val_acc /= len(val_dataloader)
        val_losses.append(current_val_loss)
        val_accuracies.append(current_val_loss)

        # Print loss and accuracy
        print(f"Epoch [{epoch+1}/{num_epochs}], "
              f"Train Loss: {current_train_loss:.4f}, Train Acc: {current_train_acc:.4f}, "
              f"Val Loss: {current_train_loss:.4f}, Val Acc: {current_val_acc:.4f}")

    # Return the losses and accuracies
    return train_losses, val_losses, train_accuracies, val_accuracies



Next, we'll test that your training loop is correct. A [common sanity check in deep learning](https://karpathy.github.io/2019/04/25/recipe/) is to overfit to a small dataset, like a single batch of data. This ensures that shapes and devices are correctly set, and the network can learn/memorize the training data, which is a good starting point before training on the full dataset.

In [ ]:
def test_train_loop():
    import torch.nn as nn
    import torch.optim as optim
    import torch.utils.data as data

    # init simple model, optimizer, loss_fn, dataloaders
    linear_model = nn.Linear(2, 3)
    optimizer = optim.SGD(linear_model.parameters(), lr=0.01)
    loss_fn = nn.CrossEntropyLoss()
    testing_loader_train = data.DataLoader(
        data.TensorDataset(torch.randn(2, 2), torch.randint(0, 3, (2,)))
    )
    testing_loader_val = data.DataLoader(
        data.TensorDataset(torch.randn(2, 2), torch.randint(0, 3, (2,)))
    )

    # copy parameters to check for changes
    params_before_training = list(linear_model.parameters())
    params_before_training = [p.clone().to("cpu") for p in params_before_training]

    print("Testing training loop, CPU")
    train_model(
        model=linear_model,
        optimizer=optimizer,
        loss_fn=loss_fn,
        num_epochs=100,
        train_dataloader=testing_loader_train,
        val_dataloader=testing_loader_val,
        device="cpu",
    )
    if torch.cuda.is_available():
        print("Testing training loop, CUDA")
        train_model(
            model=linear_model,
            optimizer=optimizer,
            loss_fn=loss_fn,
            num_epochs=100,
            train_dataloader=testing_loader_train,
            val_dataloader=testing_loader_val,
            device="cuda" if torch.cuda.is_available() else "cpu",
        )

    params_after_training = list(linear_model.parameters())
    params_after_training = [p.clone().to("cpu") for p in params_after_training]
    for p_before, p_after in zip(params_before_training, params_after_training):
        assert torch.any(
            p_before != p_after
        ), "Model parameters did not change during training"

    # check that we could overfit a single example
    (x, y) = next(iter(testing_loader_train))
    x.to("cpu")
    y.to("cpu")
    linear_model.to("cpu")
    assert (
        linear_model(x).argmax().item() == y.item()
    ), "Model could not overfit a single example"


def test_accuracy():
    y_pred = torch.tensor([[0.1, 0.8, 0.1], [0.3, 0.4, 0.3], [0.1, 0.1, 0.8]])
    y_true = torch.tensor([1, 0, 2])
    acc = accuracy(y_pred, y_true)
    assert isinstance(acc, torch.Tensor), "Accuracy should be a torch.Tensor"
    assert torch.isclose(acc, torch.tensor(2 / 3)), f"Accuracy is {acc}, expected 2/3"


try:
    test_train_loop()
except Exception as e:
    print(e)
    assert False, "Training test failed, see error above"

test_accuracy()

Now, create a neural network using PyTorch. You can use any architecture you want. Save the model in the variable `model`.

In [317]:
# A neural network model with 4 fully connected layers and batch normalization after each layer 
class NeuralNetwork(nn.Module):
    # Initialization of the layers in the model
    def __init__(self, input_size, num_classes):
        super(NeuralNetwork, self).__init__()
        self.fc1 = nn.Linear(input_size, 128)    
        self.bn1 = nn.BatchNorm1d(128)  # Batch normalization for the first layer
        self.fc2 = nn.Linear(128, 64)
        self.bn2 = nn.BatchNorm1d(64)   # Batch normalization for the second layer
        self.fc3 = nn.Linear(64, 32)
        self.bn3 = nn.BatchNorm1d(32)   # Batch normalization for the third layer
        self.fc4 = nn.Linear(32, num_classes)
    
    # Forward pass
    def forward(self, x):
        x = self.fc1(x)
        x = self.bn1(x)  # Apply batch normalization
        x = torch.relu(x)
        
        x = self.fc2(x)
        x = self.bn2(x)  # Apply batch normalization
        x = torch.relu(x)
        
        x = self.fc3(x)
        x = self.bn3(x)  # Apply batch normalization
        x = torch.relu(x)
        
        x = torch.softmax(self.fc4(x), dim=1)
        return x
    
# Initialize the model (input_size=2 for longitude and latitude, num_classes=3 for the 3 pokemon classes)
model = NeuralNetwork(input_size=2, num_classes=3)

# Move the model to the GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

print(model)

NeuralNetwork(
  (fc1): Linear(in_features=2, out_features=128, bias=True)
  (bn1): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (fc2): Linear(in_features=128, out_features=64, bias=True)
  (bn2): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (fc3): Linear(in_features=64, out_features=32, bias=True)
  (bn3): BatchNorm1d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (fc4): Linear(in_features=32, out_features=3, bias=True)
)


In [318]:
assert isinstance(model, nn.Module)


# check that we can run input of shape (batch_size, 2) through the model, with batch_size=16
def test_model_output_shape(model):
    is_model_on_cuda = next(model.parameters()).is_cuda
    device = torch.device("cuda" if is_model_on_cuda else "cpu")
    assert (
        len(model(torch.randn(16, 2, device=device))) == 16
    ), "The model should not change the batch size"


test_model_output_shape(model)

Train the network. 

For you to pass this assignment, you must obtain an accuracy on the test set greater than 60%. You can use the validation set as a proxy during development, but remember that they can differ slightly. We use the test set, as this better represents the performance of the model on new, unseen data (which is the ultimate goal).

To reach the level of performance, it may be necessary to search for a good architecture by trying several different ones. Last, if you want a challenge, try getting an accuracy greater than 75% (our reference solution achieves ~78%).

Again, it might be useful to plot the loss and accuracy (for training and validation) during training.

In [319]:
epochs = 200
learning_rate = 1e-3

# Define the loss function and optimizer
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

# Train the model
train_losses, val_losses, train_accuracies, val_accuracies = train_model(
    model=model,
    optimizer=optimizer,
    loss_fn=loss_fn,
    num_epochs=epochs,
    train_dataloader=loader_train,
    val_dataloader=loader_val,
    device=device,
)


/var/folders/w7/n9y0bc4n5zgb8gg118j_j66h0000gn/T/ipykernel_15008/46787403.py:23: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  acc = torch.tensor(prediction == y).float().mean()


Epoch [1/200], Train Loss: 1.0728, Train Acc: 0.5308, Val Loss: 1.0728, Val Acc: 0.3190
Epoch [2/200], Train Loss: 1.0010, Train Acc: 0.6573, Val Loss: 1.0010, Val Acc: 0.4362
Epoch [3/200], Train Loss: 0.9686, Train Acc: 0.6967, Val Loss: 0.9686, Val Acc: 0.6292
Epoch [4/200], Train Loss: 0.9488, Train Acc: 0.6743, Val Loss: 0.9488, Val Acc: 0.6439
Epoch [5/200], Train Loss: 0.9391, Train Acc: 0.6958, Val Loss: 0.9391, Val Acc: 0.5883
Epoch [6/200], Train Loss: 0.9201, Train Acc: 0.6959, Val Loss: 0.9201, Val Acc: 0.6020
Epoch [7/200], Train Loss: 0.9151, Train Acc: 0.7015, Val Loss: 0.9151, Val Acc: 0.6722
Epoch [8/200], Train Loss: 0.9095, Train Acc: 0.7010, Val Loss: 0.9095, Val Acc: 0.6790
Epoch [9/200], Train Loss: 0.8976, Train Acc: 0.7225, Val Loss: 0.8976, Val Acc: 0.6790
Epoch [10/200], Train Loss: 0.8824, Train Acc: 0.7225, Val Loss: 0.8824, Val Acc: 0.6761
Epoch [11/200], Train Loss: 0.8883, Train Acc: 0.7206, Val Loss: 0.8883, Val Acc: 0.6751
Epoch [12/200], Train Loss: 0.

Let's have a look at what we have learned! Create a function that visualizes the decision regions of the network. Overlap it with the points corresponding to the training data and validations data, such as in Section 2, by using the scatter plot function. The training and validation points should have different markers and/or colors. Last, call the function to visualize the decision regions of your network.

Hint: A simple way to show the decision region is to generate a lot of points within a predefined range of longitude and latitude and apply your network to it. However, feel free to explore other ways.

In [256]:
def plot_decision_region(model: nn.Module, x_train, y_train, x_val, y_val):
    """
    Plot the decision region of the model.

    Args:
    model: nn.Module: The trained model
    x_train: The training features, use whatever representation you prefer (e.g. numpy array, torch.Tensor, dataset, dataloader)
    y_train: The training labels, use whatever representation you prefer (e.g. numpy array, torch.Tensor, dataset, dataloader)
    x_val: The validation features, use whatever representation you prefer (e.g. numpy array, torch.Tensor, dataset, dataloader)
    y_val: The validation labels, use whatever representation you prefer (e.g. numpy array, torch.Tensor, dataset, dataloader)
    """
    # YOUR CODE HERE
    # Determine the range for the grid
#     x_min, x_max = x_train[:, 1].min() - 1, x_train[:, 1].max() + 1
#     y_min, y_max = x_train[:, 0].min() - 1, x_train[:, 0].max() + 1
    
#     # Create a grid of points
#     xx, yy = np.meshgrid(np.linspace(x_min, x_max, 200),
#                          np.linspace(y_min, y_max, 200))
    
#     # Flatten the grid to pass through the model
#     grid_points = np.c_[xx.ravel(), yy.ravel()]
    
#     # Use the model to predict on the grid points
#     model.eval()
#     with torch.no_grad():
#         predictions = model(torch.tensor(grid_points, dtype=torch.float32))
#         predicted_classes = torch.argmax(predictions, dim=1).numpy()
    
#     # Reshape the predictions to match the grid shape
#     predicted_classes = predicted_classes.reshape(xx.shape)
    
#     cmap_light = ListedColormap(['#FFAAAA', '#AAFFAA', '#AAAAFF'])
#     cmap_bold = ListedColormap(['#FF0000', '#00FF00', '#0000FF'])

#     # Plot the decision regions
#     plt.figure(figsize=(10, 6))
#     plt.contourf(xx, yy, predicted_classes, alpha=0.3, cmap=cmap_light)
    
#     # Overlay the training and validation data points
#     plt.scatter(x_train[:, 1], x_train[:, 0], c=y_train, cmap=cmap_bold, edgecolor='k', marker='o', label='Train Data')
#     plt.scatter(x_val[:, 1], x_val[:, 0], c=y_val, cmap=cmap_bold, edgecolor='k', marker='s', label='Validation Data')
    
#     # Mapping of classes to names
#     pokemon_mapping = {0: 'Diglett', 1: 'Seel', 2: 'Tauros'}
    
#     # Add custom legend for decision boundaries
#     boundary_colors = ['#FFAAAA', '#AAFFAA', '#AAAAFF']
#     boundary_patches = [mpatches.Patch(color=boundary_colors[i], label=f'{pokemon_mapping[i]}') for i in range(len(boundary_colors))]
    
#     # Add labels, legend, and title
#     plt.xlabel('Longitude')
#     plt.ylabel('Latitude')
#     plt.legend(handles=boundary_patches + [plt.Line2D([0], [0], marker='o', color='w', markerfacecolor='black', markersize=10, label='Train Data'),
#                                            plt.Line2D([0], [0], marker='s', color='w', markerfacecolor='black', markersize=10, label='Validation Data')])
#     plt.title('Decision Regions with Training and Validation Data')
#     plt.show()

# # Call the function
# plot_decision_region(model, x_train, y_train, x_val, y_val)
    

Do the learned decision regions look like you would expect? Can they be improved? In what sense, and how would that change your model? Please comment on your results. 


**Your answer:** (fill in here)

### 4.1. Model capacity and generalization

Now we have all the neccessary tools to do a small experiment on model capacity and implications on generalization.

Begin by defining a neural network `tiny_model` with a single linear layer and appropriate activation function. Then, train the network until convergence (should be fast).

In [314]:
# YOUR CODE HERE
class tiny_model(nn.Module):
    def __init__(self, input_size, num_classes):
        super(tiny_model, self).__init__()
        self.fc1 = nn.Linear(input_size, num_classes)

    def forward(self, x):
        x = torch.softmax(self.fc1(x), dim=1)
        return x

tiny_model = tiny_model(input_size=2, num_classes=3)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
tiny_model.to(device)

tiny_model(
  (fc1): Linear(in_features=2, out_features=3, bias=True)
)

In [315]:
assert isinstance(tiny_model, nn.Module), "tiny_model should be a torch.nn.Module"
test_model_output_shape(tiny_model)

Now, draw the decision regions of the network as you did in the previous section. Before running the code, think about what you expect to see. What will the regions look like? How will they differ from the previous ones?

In [316]:
# YOUR CODE HERE
# Define the loss function and optimizer
epochs = 100
learning_rate = 1e-3
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(tiny_model.parameters(), lr=learning_rate)

train_losses, val_losses, train_accuracies, val_accuracies = train_model(
    model=tiny_model,
    optimizer=optimizer,
    loss_fn=loss_fn,
    num_epochs=epochs,
    train_dataloader=loader_train,
    val_dataloader=loader_val,
    device=device,
)

# plot_decision_region(tiny_model, x_train, y_train, x_val, y_val)

/var/folders/w7/n9y0bc4n5zgb8gg118j_j66h0000gn/T/ipykernel_15008/46787403.py:23: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  acc = torch.tensor(prediction == y).float().mean()


Epoch [1/100], Train Loss: 1.0622, Train Acc: 0.5082, Val Loss: 1.0622, Val Acc: 0.4137
Epoch [2/100], Train Loss: 1.0594, Train Acc: 0.5061, Val Loss: 1.0594, Val Acc: 0.4128
Epoch [3/100], Train Loss: 1.0584, Train Acc: 0.5033, Val Loss: 1.0584, Val Acc: 0.4128
Epoch [4/100], Train Loss: 1.0568, Train Acc: 0.4991, Val Loss: 1.0568, Val Acc: 0.4128
Epoch [5/100], Train Loss: 1.0535, Train Acc: 0.5117, Val Loss: 1.0535, Val Acc: 0.4128
Epoch [6/100], Train Loss: 1.0546, Train Acc: 0.5090, Val Loss: 1.0546, Val Acc: 0.4128
Epoch [7/100], Train Loss: 1.0547, Train Acc: 0.5020, Val Loss: 1.0547, Val Acc: 0.4118
Epoch [8/100], Train Loss: 1.0504, Train Acc: 0.5104, Val Loss: 1.0504, Val Acc: 0.4118
Epoch [9/100], Train Loss: 1.0529, Train Acc: 0.5085, Val Loss: 1.0529, Val Acc: 0.4098
Epoch [10/100], Train Loss: 1.0533, Train Acc: 0.4975, Val Loss: 1.0533, Val Acc: 0.4079
Epoch [11/100], Train Loss: 1.0532, Train Acc: 0.4920, Val Loss: 1.0532, Val Acc: 0.4059
Epoch [12/100], Train Loss: 1.

Are the decision regions any different? Why/why not? What does this tell you about the model capacity and generalization?

**Your answer:** (fill in here)

Next, create a neural network `large_model` with many layers and/or hidden units. Try to maximize the training accuracy (0.7-0.8). Getting these models to converge might be a bit tricky, so you might have to adjust the learning rate, the optimizer, etc. Also, might be easier to have a wide model (large number of hidden units, say 1024 per layer) than a deep model (more than 4 layers).

Feel free to experiment, but if you get stuck, our model is a MLP with an input layer, 4 hidden layers with 1024 units each, and an output layer. We used the ReLU activation function between each layer and a softmax for the output layer. We used the Adam optimizer with a learning rate of 0.003 and a batch size of 512. We trained for at least a few hundred epochs. Also, normalizing the input can be helpful (zero mean and unit variance).

In [342]:
# YOUR CODE HERE
class large_model(nn.Module):
    def __init__(self, input_size, num_classes):
        super(large_model, self).__init__()
        self.fc1 = nn.Linear(input_size, 512) 
        self.fc2 = nn.Linear(512, 512)
        self.fc3 = nn.Linear(512, 512)
        self.fc4 = nn.Linear(512, 512)
        self.fc5 = nn.Linear(512, num_classes)
    
    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = torch.relu(self.fc3(x))
        x = torch.relu(self.fc4(x))
        x = torch.softmax(self.fc5(x), dim=1)
        return x 
    
large_model = large_model(input_size=2, num_classes=3)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
large_model.to(device) 

large_model(
  (fc1): Linear(in_features=2, out_features=512, bias=True)
  (fc2): Linear(in_features=512, out_features=512, bias=True)
  (fc3): Linear(in_features=512, out_features=512, bias=True)
  (fc4): Linear(in_features=512, out_features=512, bias=True)
  (fc5): Linear(in_features=512, out_features=3, bias=True)
)

In [343]:
assert isinstance(large_model, nn.Module), "large_model should be a torch.nn.Module"
test_model_output_shape(large_model)

Again, draw the decision regions of the network. What do you expect to see now? How will the regions differ from the previous ones?

In [344]:
# YOUR CODE HERE
epochs = 200
learning_rate = 0.002
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(large_model.parameters(), lr=learning_rate)

train_losses, val_losses, train_accuracies, val_accuracies = train_model(
    model=large_model,
    optimizer=optimizer,
    loss_fn=loss_fn,
    num_epochs=epochs,
    train_dataloader=loader_train,
    val_dataloader=loader_val,
    device=device,
)

# def plot_results(train_losses, val_losses, train_accs, val_accs):
#   epochs = range(1, len(train_losses) + 1)
  
#   plt.figure(figsize=(17, 6))
  
#   # Plotting Loss
#   plt.subplot(1, 2, 1)
#   plt.plot(epochs, train_losses, label='Training Loss')
#   plt.plot(epochs, val_losses, label='Validation Loss')
#   plt.xlabel('Epochs')
#   plt.ylabel('Loss')
#   plt.title('Training and Validation Loss')
#   plt.legend()
  
#   # Plotting Accuracy
#   plt.subplot(1, 2, 2)
#   plt.plot(epochs, train_accs, label='Training Accuracy')
#   plt.plot(epochs, val_accs, label='Validation Accuracy')
#   plt.xlabel('Epochs')
#   plt.ylabel('Accuracy')
#   plt.title('Training and Validation Accuracy')
#   plt.legend()
  
#   # plt.tight_layout()
#   plt.show()

# plot_results(train_losses, val_losses, train_accuracies, val_accuracies)
# plot_decision_region(large_model, x_train, y_train, x_val, y_val)


/var/folders/w7/n9y0bc4n5zgb8gg118j_j66h0000gn/T/ipykernel_15008/46787403.py:23: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  acc = torch.tensor(prediction == y).float().mean()


Epoch [1/200], Train Loss: 1.0572, Train Acc: 0.4504, Val Loss: 1.0572, Val Acc: 0.4810
Epoch [2/200], Train Loss: 0.9336, Train Acc: 0.5873, Val Loss: 0.9336, Val Acc: 0.5825
Epoch [3/200], Train Loss: 0.8910, Train Acc: 0.6603, Val Loss: 0.8910, Val Acc: 0.5932
Epoch [4/200], Train Loss: 0.8558, Train Acc: 0.6752, Val Loss: 0.8558, Val Acc: 0.6546
Epoch [5/200], Train Loss: 0.8693, Train Acc: 0.6740, Val Loss: 0.8693, Val Acc: 0.5893
Epoch [6/200], Train Loss: 0.8778, Train Acc: 0.6672, Val Loss: 0.8778, Val Acc: 0.5961
Epoch [7/200], Train Loss: 0.8501, Train Acc: 0.6967, Val Loss: 0.8501, Val Acc: 0.5971
Epoch [8/200], Train Loss: 0.8766, Train Acc: 0.6658, Val Loss: 0.8766, Val Acc: 0.6868
Epoch [9/200], Train Loss: 0.8360, Train Acc: 0.7096, Val Loss: 0.8360, Val Acc: 0.6868
Epoch [10/200], Train Loss: 0.8153, Train Acc: 0.7370, Val Loss: 0.8153, Val Acc: 0.6245
Epoch [11/200], Train Loss: 0.8130, Train Acc: 0.7395, Val Loss: 0.8130, Val Acc: 0.6790
Epoch [12/200], Train Loss: 0.

How do the decision regions differ between the tiny, large and the network you trained? Can you explain why this happens? Relate your answer to the concepts you learned in the first lectures.

**Your answer:** (fill in here)

## 5. Evaluation

Back to your original model. Once you achieved at least 60% accuracy in the validation set with your main model, we are done with its training. Now we'll evaluate the performance of your classifier on the test set.

Compute the accuracy on the test set.

In [ ]:
main_model_test_accuracy = 0
# YOUR CODE HERE
print(f"Test accuracy: {main_model_test_accuracy:.3f}.")

In [ ]:
assert main_model_test_accuracy > 0.6

Most likely, you'll get a different (slightly worse) accuracy than the one you got on the validation set. Why is this? Also, why do we need both a test and validation set?

**Your answer:** (fill in here)

Next, compute the confusion matrix of your predictions on the test set and save it as `conf_mat`.

In [ ]:
# YOUR CODE HERE
print(conf_mat)

In [ ]:
assert isinstance(conf_mat, np.ndarray) or isinstance(
    conf_mat, torch.Tensor
), "conf_mat should be a numpy array or torch.Tensor"
assert conf_mat.shape == (3, 3), "conf_mat should have shape (3, 3), i.e. 3 classes"
assert np.sum(conf_mat) == len(
    dataset_test
), "conf_mat should sum up to the number of test samples"

What can you conclude from the computed accuracy and confusion matrix?

**Your answer:** (fill in here)

## 6. Exploration

You have now trained and evaluated a neural network for this particular classification task. Can you provide a brief explanation as to how you could use it to decide where to travel, if you're interested in capturing the aforementioned Pokemons?

**Your answer:** (fill in here)

Is (are) there any other feature(s) from the original dataset (e.g. hour of the day, pressure, wind speed, population density, etc.) which you think would be valuable to add as an input feature to your classifier to improve its performance? 

**Your answer:** (fill in here)

To investigate your hypothesis, plot a histogram of the selected feature(s) for each one of the pokemons we're interested in. For example, if you think pressure and population density are valuable for prediction, plot 6 histograms. 3 of them will be the pressure histograms for each class ('Diglett', 'Seel' and 'Tauros'), and the other 3 will be the population density for each class.

In [ ]:
# YOUR CODE HERE

What does/do this/these histogram(s) show you? Could it be beneficial to add this/these new feature(s) as input? Explain why/why not.

**Your answer:** (fill in here)

The purpose was this assignment was mostly to make you design a network for classification, using this Pokemon dataset as use case. However, if you want to find those three particular Pokemons, most likely using a network for classification is not the best approach. An alternative would be to perform localization by using regression instead. **Can you state some pros and cons of approach this as a regression problem instead of a classification problem?** (We do not expect very detailed answers, you will pass the assignment as long as you make a reasonable attempt at explaining the pros and cons.)

**Your answer:** (fill in here)

## 7. (optional)

Assuming you found useful new features in the last part of this assignment, train a new classifier that uses these featues as well. Did the accuracy on the validation set improve? What's the highest accuracy you can achieve?

In [ ]:
# YOUR CODE HERE